## Settings

In [1]:
import datetime
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

# Loading the data

In [2]:
df = pd.read_csv("/Users/quantic/Desktop/sp_dataset.csv")

# Data cleaning - Section 1

1. The data provided is dirty explain what steps should be taken to clean it, and provide example code that performs the cleaning operations you have specified.

## Data exploration

In [3]:
# check the size of the dataset
print(f'The dataset contains   {df.shape[0]:,}   rows and   {df.shape[1]}   columns')

The dataset contains   36,112   rows and   26   columns


In [4]:
# check each columns names, count Non-null and data types
df.info()

# Note: A few columns have Null values which can be an indicator of tracking issues, 
#       namely: referral_url, geo columns (3x), page_title, link_click_target_url

# Note 2: Page_title & link_click_target_url null values is per design when a link_click event is recorded

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36112 entries, 0 to 36111
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   event_id               36112 non-null  object 
 1   web_page_id            36112 non-null  object 
 2   session_id             36112 non-null  object 
 3   session_index          36112 non-null  int64  
 4   user_cookie            36112 non-null  object 
 5   event_name             36112 non-null  object 
 6   page_title             34083 non-null  object 
 7   page_urlhost           36112 non-null  object 
 8   page_url               36112 non-null  object 
 9   referral_url           20205 non-null  object 
 10  dvce_created_tstamp    36112 non-null  object 
 11  geo_country            35048 non-null  object 
 12  geo_region             29157 non-null  object 
 13  geo_city               29096 non-null  object 
 14  geo_timezone           33291 non-null  object 
 15  pp

In [5]:
# Check the first 100 rows
#df.head(100)

In [6]:
# Check descriptive statistics of numeric fields
df.describe()

,session_index,pp_xoffset_min,pp_xoffset_max,pp_yoffset_min,pp_yoffset_max,doc_width,doc_height
count,36112.00,28518.00,28518.00,28518.00,28518.00,36112.00,36112.00
mean,27.57,0.20,0.59,1738.84,2340.59,1443.27,6843.93
std,109.88,8.26,10.81,2336.05,2628.61,400.18,4373.77
min,1.00,-245.00,0.00,-188.00,0.00,238.00,405.00
25%,1.00,0.00,0.00,0.00,423.00,1280.00,3012.00
50%,2.00,0.00,0.00,900.00,1500.00,1425.00,5862.00
75%,7.00,0.00,0.00,2487.00,3280.00,1654.00,11498.00
max,1190.00,387.00,398.00,22674.00,30494.00,3423.00,104642.00


In [7]:
# Check dates columns
print(f'Date ranges:  {min(pd.to_datetime(df.dvce_created_tstamp))} and {max(pd.to_datetime(df.dvce_created_tstamp))}')

Date ranges:  2020-02-10 04:11:00 and 2021-12-02 23:59:00


# List of Data Problems:

1. pp_xoffset_min contains negative numbers - it should be impossible.
2. pp_yoffset_min contains negative numbers - it should be impossible.
3. Drop irrelevant data in Referral_url e.g. 'XXX'
4. Geo_region contains data inconsistency with a mixture of string and integer. Additionally, data in geo_region doesn't match geo_country e.g. for geo_country = US, geo_region = CA
5. Data inconsistency with UTC values in os_timezone
6. Device_family needs further data transformation to be actionable
7. Investigate the data collection process to understand if missing values in referral_url, geo columns (3x) is intended. NB: For geo_columns, the os_timezone field has no missing values and can be used to create similar splits.

### Solution 1 & 2:
* pp_xoffset_min contains negative numbers
* pp_yoffset_min contains negative numbers

In [8]:
# pp_xoffset_min replace negative numbers by 0 (inplace)
df.loc[df['pp_xoffset_min'] < 0, ['pp_xoffset_min']] = 0
# pp_yoffset_min replace negative numbers by 0 (inplace)
df.loc[df['pp_yoffset_min'] < 0, ['pp_yoffset_min']] = 0
# check if change was implemented
print(f"pp_xoffset_min :  \n{min(df['pp_xoffset_min'])}", end='\n\n')
print(f"pp_yoffset_min :  \n{min(df['pp_yoffset_min'])}")

pp_xoffset_min :  
0.0

pp_yoffset_min :  
0.0


### Solution 3:
* Drop irrelevant data in Referral_url e.g. 'XXX'

In [9]:
# Data exploration
# df.loc[df['referral_url'] == 'XXX']

In [10]:
# SOLUTION: replace XXX by NaN
df['referral_url'] = df.loc[df['referral_url'] != 'XXX', ['referral_url']]
# check if change is implemented correctly
print(f"referral_url unique values :  \n\n{list(df['referral_url'].unique())}")

referral_url unique values :  

[nan, 'discourse.snowplowanalytics.com', 'www.google.com', 'snowplowanalytics.com', 'www.chess.com', 'yandex.ru', 'duckduckgo.com', 'github.com', 'www.aol.de']


In [11]:
# check if change is correctly implemented
df.iloc[72]

event_id                              5b562581-3ce5-470f-b62e-ea00b55ed326
web_page_id                           4420373e-ada9-4521-bcc4-6b30cca2db47
session_id               bc866191-22a4-4830-8f5a-9e38253fae4f          ...
session_index                                                            1
user_cookie                           bd393077-0949-422b-a21f-1e9b7593c6e1
event_name                                                       page_ping
page_title               Dealing with Hadoop's small files problem – Sn...
page_urlhost                                         snowplowanalytics.com
page_url                 https://snowplowanalytics.com/blog/2013/05/30/...
referral_url                                                           NaN
dvce_created_tstamp                                       11/02/2021 07:21
geo_country                                                             US
geo_region                                                              CA
geo_city                 

### Solution 4:
* Geo_region contains data inconsistency with a mixture of string and integer. Additionally, data in geo_region doesn't match geo_country e.g. for geo_country = US, geo_region = CA

In [12]:
# Data exploration: Unique values in geo_region 
print(list(df.geo_region.unique()))

# Note: 
# Data inconsistency with a mixture of string and integer. 
# Additionally, data in geo_region doesn't match geo_country e.g. for geo_country = US, geo_region = CA

['CA', '7', '2', '16', '8', '0', nan, 'NY', 'TX', 'VA', '39', '12', 'NM', '10', '23', 'WA', 'NJ', 'J8', '56', '60', 'A8', '5', '4', '72', '36', 'H1', '19', 'H9', '20', '35', '26', '22', '58', 'K5', '25', '6', '17', 'MD', 'CO', 'FL', '42', 'MA', 'C3', 'IN', 'NH', '48', '11', 'IL', 'GA', 'NC', 'D9', '78', 'MN', 'OH', '9', 'AZ', 'SC', '18', 'J1', '86', '13', 'KY', '83', '77', '47', 'B8', 'KS', '33', 'B5', '44', '27', '30', '14', '3', 'H7', '75', 'E7', '55', 'B7', 'CT', 'I4', 'K2', 'B9', 'ON', 'BC', '29', 'MB', 'PA', 'OR', 'OK', 'IA', '1', '40', '84', '32', 'I7', 'G1', 'P1', 'B4', 'MO', 'DC', '21', 'WI', 'HI', 'E2', 'D7', '66', 'AB', 'I2', 'TN', '62', 'X5', 'F8', 'A5', 'ME', 'RI', 'UT', '34', '37', '53', 'G4', 'M2', '91', 'QC', 'J7', 'M5', '73', 'NV', 'MI', 'A7', 'A2', '28', 'F1', 'DE', '65', '52', 'B3', 'G5', 'P5', 'D8', 'P2', 'G2', '24', '90', 'F3', 'L1', 'NB', 'C5', '15', 'I8', 'I1', 'H5', 'P3', 'A9', '82', 'A6', 'K7', 'D4', '81', 'M3', 'ID', 'F2', 'L5', 'P9', 'N5', 'K8', '41', 'B2', '4

In [13]:
# SOLUTION: Use geo_timezone to repopulate correctly geo_region
df['geo_region'] = df['geo_timezone'].str.split('/').str[0]

In [14]:
# Check if change is correctly implemented
print(list(df.geo_region.unique()), end='\n')

['America', 'Australia', 'Asia', nan, 'Europe', 'Africa', 'Pacific', 'Atlantic', 'Indian']


### Solution 5:
* Data inconsistency with UTC values in os_timezone

In [15]:
# Data exploration

# Note: 
# US & MA are not UTC time
df.loc[(df.os_timezone == 'UTC'), ['geo_timezone', 'geo_country', 'geo_timezone']].drop_duplicates()

,geo_timezone,geo_country,geo_timezone
4,America/Los_Angeles,US,America/Los_Angeles
952,Europe/London,GB,Europe/London
6887,Atlantic/Reykjavik,IS,Atlantic/Reykjavik
9972,America/New_York,US,America/New_York
10554,Africa/Casablanca,MA,Africa/Casablanca
17586,America/Denver,US,America/Denver
27436,Africa/Accra,GH,Africa/Accra


In [16]:
# SOLUTION: overwrite UTC with proper geo_timezone
df.loc[(df.os_timezone == 'UTC') & (df.geo_country == 'US'), ['os_timezone']] = df.geo_timezone
# check if change is correctly implemented
df.loc[(df.os_timezone == 'UTC') & (df.geo_country == 'US'), ['os_timezone']].count()

os_timezone    0
dtype: int64

In [17]:
# SOLUTION: overwrite UTC with proper geo_timezone
df.loc[(df.os_timezone == 'UTC') & (df.geo_country == 'MA'), ['os_timezone']] = df.geo_timezone
# check if change is correctly implemented
df.loc[(df.os_timezone == 'UTC') & (df.geo_country == 'MA'), ['os_timezone']].count()

os_timezone    0
dtype: int64

### Solution 6:

Device_family needs further data transformation to be actionable. The field cannot be used currently. I advise to create another column grouping each value according to their brand e.g. SM-G930S should be classified as Samsung

NB: This is a very manual process as I would need to google each entry in the list below. In the interest of time and because I won't use this field in the next section of this analysis, I will skip the implementation of the solution proposed here.

In [18]:
df.device_family.value_counts()

Other                      33337
iPhone                       748
Spider                       500
iPad                         196
SAMSUNG-SM-G930A             136
Nexus 5X                     127
Nexus 6P                      79
MI 5                          75
SM-G930F                      72
SM-G900F                      63
Moto G (4)                    61
MotoG3                        57
XT1032                        37
XT1254                        36
Redmi Note 3                  35
XT1706                        34
SAMSUNG SM-J500F              30
SM-J700F                      30
XT1068                        23
SM-N750                       21
A0001                         21
ASUS_Z017D                    20
SM-G920T                      20
Redmi Note 4                  19
LG-H812                       18
SM-G920F                      18
HUAWEI ALE-L04                17
SM-G930S                      16
SM-G900T                      15
SM-G935F                      14
SM-G930T  

# Final Dataset

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36112 entries, 0 to 36111
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   event_id               36112 non-null  object 
 1   web_page_id            36112 non-null  object 
 2   session_id             36112 non-null  object 
 3   session_index          36112 non-null  int64  
 4   user_cookie            36112 non-null  object 
 5   event_name             36112 non-null  object 
 6   page_title             34083 non-null  object 
 7   page_urlhost           36112 non-null  object 
 8   page_url               36112 non-null  object 
 9   referral_url           19491 non-null  object 
 10  dvce_created_tstamp    36112 non-null  object 
 11  geo_country            35048 non-null  object 
 12  geo_region             33291 non-null  object 
 13  geo_city               29096 non-null  object 
 14  geo_timezone           33291 non-null  object 
 15  pp

# Data analysis & modelling - Section 2

2. Use the clean dataset to calculate the following metrics (share any code generated when calculating these metrics):
   * a. Session bounce rate
   * b. Average time engaged on each page view
   * c. Average scroll depth on each page view
   * d. What % of users start their journey on page_urlhost ='snowplowanalytics.com' before navigating to page_urlhost ='discourse.snowplowanalytics.com'

## Compute:
* a. Session Bounce Rate

In [20]:
# DEFINTION of bounce rate: 
# percentage of visitors to a particular website who navigate away from the site after viewing only one page

# Count page_views per session
df_session_count = (df.loc[
                          df['event_name']=='page_view'
                          ,['session_id','event_name']
                          ].groupby(['session_id'])
                           .count()
                   )
# Session bounce rate = sessions with 1 page view / total sessions count
session_bounce_rate = len(df_session_count.loc[df_session_count['event_name'] == 1])/len(df_session_count)
# print results
print(f'Session bounce rate is {session_bounce_rate:.2%}')

Session bounce rate is 74.27%


## Compute:
* b. Average time engaged on each page view

In [21]:
# Filter dataset on page_view & page_ping + select only relevant columns
df_time = (df.loc[
                 (df['event_name']=='page_view') | (df['event_name']=='page_ping')
                 ,['session_id','page_title','dvce_created_tstamp']
                 ]
          )
# convert to datetime
df_time['dvce_created_tstamp'] = pd.to_datetime(df_time['dvce_created_tstamp'])
# add min & max columns
df_time['max'] = df_time.groupby(['session_id','page_title'])['dvce_created_tstamp'].transform('max')
df_time['min'] = df_time.groupby(['session_id','page_title'])['dvce_created_tstamp'].transform('min')
# drop irrelevant column
df_time.drop(['dvce_created_tstamp'], axis=1, inplace=True)
# add time on site column
df_time['time_on_site'] = df_time.apply(
                                        lambda row: row['max'] - row['min'] 
                                        if row['max'].date() == row['min'].date() 
                                        else pd.to_timedelta(0)
                                        , axis=1
                                        )
# convert timedelta to seconds
df_time['time_on_site'] = df_time['time_on_site'].apply(lambda row: row.total_seconds())
# convert seconds to minutes
df_time['time_on_site'] = df_time.apply(lambda row: row['time_on_site']/60, axis=1)
# compute the average time on site per page title
avg_time= df_time.groupby(['page_title']).mean().sort_values(by='time_on_site',ascending=False)
# convert float to percentage for file export
df_time['time_on_site'] = df_time['time_on_site'].apply(lambda x: format(x, '.2'))

In [22]:
# Average time on each page in minutes
avg_time.head(10)

,time_on_site
page_title,
User id and session id in AMP page - Tracking SDKs - Discourse – Snowplow,856.00
How to store kinesis enriched stream to redshift - Storage targets - Discourse – Snowplow,431.00
Snowplow Bounce Rate - Tableau - For data modelers & consumers - Discourse – Snowplow,289.22
Batch v real time enrichment - Discourse – Snowplow,223.85
Why is the Redshift table definition for a schema not the latest version? - For engineers / Storage targets - Discourse – Snowplow,189.00
Enriched event stream into Redshift using StorageLoader: Contract violation error - For engineers / Storage targets - Discourse – Snowplow,182.08
How to store kinesis enriched stream to redshift - For engineers / Storage targets - Discourse – Snowplow,173.00
Error using StorageLoader to load data into Redshift - For data modelers & consumers / Redshift - Discourse – Snowplow,136.04
Debugging bad rows in Spark and Zeppelin [tutorial] - For data modelers & consumers - Discourse – Snowplow,102.85


In [23]:
# check shape
avg_time.shape

(717, 1)

In [24]:
# Data consistency check: unique number of page_titles in original dataset. 717 + 1 (NaN)
len(pd.unique(df['page_title']))

718

In [25]:
# Export full table with 717 unique pages
avg_time.to_csv("/Users/quantic/Desktop//Snowplow - Use Case/Task Two - 2a - avg time per page.csv")

## Compute:
* c. Average time scroll depth on each page view

In [26]:
# Filter dataset on page_view & page_ping + select only relevant columns
df_scroll = df.loc[
                    (df['event_name']=='page_view') | (df['event_name']=='page_ping')
                    ,['page_title','pp_yoffset_max','doc_height']
                  ]
# fill na
df_scroll['pp_yoffset_max'] = df_scroll['pp_yoffset_max'].fillna(0)
# add scroll depth
df_scroll['scroll_depth'] = df_scroll.apply(
                                            lambda row: row['pp_yoffset_max']/row['doc_height'] 
                                            if row['pp_yoffset_max'] != 0 
                                            else 0
                                            , axis=1
                                            )
# drop irrelevant columns
df_scroll.drop(['pp_yoffset_max','doc_height'],inplace=True, axis=1)
# set max scroll_depth to 100%
df_scroll.loc[df_scroll['scroll_depth'] > 1,['scroll_depth']] = 1
# avg scroll depth per page 
avg_scroll = df_scroll.groupby(['page_title']).mean().sort_values(by='scroll_depth',ascending=False)
# convert float to percentage 
avg_scroll['scroll_depth'] = avg_scroll['scroll_depth'].apply(lambda x: format(x, '.2%'))

In [27]:
# check if we have the same number of page_title 
# than in unique number of page_titles in original dataset. 717 + 1 (NaN)
avg_scroll.shape

(717, 1)

In [28]:
avg_scroll.to_csv("/Users/quantic/Desktop/Snowplow - Use Case/Task Two - 2b - avg scroll depth per page.csv")
avg_scroll.head(10)

,scroll_depth
page_title,
EMR jobflow failing on Hadoop Enrich step after a few seconds - For engineers / AWS batch pipeline - Discourse – Snowplow,100.00%
"Data collection: the essential but unloved, foundation of the data value chain - For data modelers & consumers - Discourse – Snowplow",100.00%
Latest For engineers topics - Discourse – Snowplow,80.67%
User id and session id in AMP page - For engineers / Tracking SDKs - Discourse – Snowplow,65.86%
Stream Collector process dead randomly - For engineers / Collectors - Discourse – Snowplow,65.12%
Error while uploading snowplow logs to s3 - For engineers / Data store sources - Discourse – Snowplow,63.04%
Processing logs for a specific time period - For engineers / AWS batch pipeline - Discourse – Snowplow,62.35%
Vacuums in Redshift and new Redshift Enhancements - For data modelers & consumers / Redshift - Discourse – Snowplow,62.30%
Latest For data modelers & consumers topics - Discourse – Snowplow,55.94%


## Compute:
* d. What % of users start their journey on page_urlhost ='snowplowanalytics.com' before navigating to page_urlhost ='discourse.snowplowanalytics.com'

### Option 1: using referral_url
Assumption: referral_url records the previous page visited

In [248]:
# Data exploration for numerator: users starting on Snowplowanalytics and continuing to Discourse
(df.loc[
        (df['event_name']=='page_view') & (df['page_urlhost']=='discourse.snowplowanalytics.com')
       ].referral_url
        .fillna('NaN')
        .value_counts()
)

www.google.com                     1043
NaN                                 348
discourse.snowplowanalytics.com     121
snowplowanalytics.com                20
www.chess.com                         3
yandex.ru                             3
duckduckgo.com                        3
github.com                            2
Name: referral_url, dtype: int64

In [251]:
# Filtering on page_urlhost & referral_url to get distinct users starting on Snowplowanalytics 
# and continuing to Discourse

# Note: I assume that referral_url records the previous page visited
discourse_page = (df.loc[
                         (df['event_name']=='page_view') 
                         & (df['page_urlhost']=='discourse.snowplowanalytics.com') 
                         & (df['referral_url']=='snowplowanalytics.com')
                         ]
                 )
discourse_page = len(discourse_page.groupby(['session_id'])['session_id'])

In [252]:
# Data exploration for denominator: users starting on Snowplowanalytics
(df.loc[
        (df['event_name']=='page_view') 
        & (df['page_urlhost']=='snowplowanalytics.com')
       ].referral_url
        .fillna('NaN')
        .value_counts()
)
# Note: I assume that referral_url NaN means no previous page_view. 
#       Thus I'll consider all users except from snowplowanalytics and discourse.


NaN                                1775
snowplowanalytics.com              1499
www.google.com                      677
yandex.ru                            13
www.chess.com                         9
github.com                            7
duckduckgo.com                        6
www.aol.de                            4
discourse.snowplowanalytics.com       2
Name: referral_url, dtype: int64

In [253]:
# Filering on page_urlhost & referral_url to get distinct users starting on Snowplowanalytics
snowplow_page = (df.loc[
                        (df['event_name']=='page_view') 
                        & (df['page_urlhost']=='snowplowanalytics.com') 
                        & (df['referral_url']!='snowplowanalytics.com') 
                        & (df['referral_url']!='discourse.snowplowanalytics.com')
                        ]
                )
snowplow_page = len(snowplow_page.groupby(['session_id'])['session_id'])

In [254]:
percent_users = discourse_page/snowplow_page
print(f'Distinct users starting on Snowplowanalytics: {snowplow_page}')
print(f'Distinct users starting on Snowplowanalytics and continuing to Discourse: {discourse_page}')
print(f'Percentage of users is: {percent_users:.2%}')

Distinct users starting on Snowplowanalytics: 2004
Distinct users starting on Snowplowanalytics and continuing to Discourse: 18
Percentage of users is: 0.90%


### Option 2: ranking page_view events per session
Assumption: 

In [255]:
# Filter data on page view + retrieving relevant columns
df_temp = df.loc[(df['event_name']=='page_view'), ['session_id','page_urlhost','dvce_created_tstamp','referral_url']].reset_index(drop=True)
# converting dvce_created_tstamp to timestap
df_temp['dvce_created_tstamp'] = pd.to_datetime(df_temp['dvce_created_tstamp'])
# sorting values by session_id & timestamp
df_temp.sort_values(by=['session_id','dvce_created_tstamp'],ascending=True,inplace=True)
# ranking page_views for each session by their time appearance
df_temp['rank'] = df_temp.groupby(['session_id'])['dvce_created_tstamp'].rank(method='first')

In [263]:
# Create a table with all users starting with snowplowanalytics.com
df_table1 = df_temp.loc[(df_temp['page_urlhost']=='snowplowanalytics.com')&(df_temp['rank']==1)&(df_temp['referral_url']!='snowplowanalytics.com')]
# count distinct on session_id
denominator = len(df_table1.groupby(['session_id'])['session_id'])
len(df_table1.groupby(['session_id'])['session_id'])

1868

In [260]:
#df_table1.referral_url.value_counts()
df_table1.loc[df_table1['referral_url']=='snowplowanalytics.com']

,session_id,page_urlhost,dvce_created_tstamp,referral_url,rank
2776,009847f5-0018-4ccf-8098-363980188f8e ...,snowplowanalytics.com,2021-02-14 00:18:00,snowplowanalytics.com,1.00
4052,00c0baf0-0020-44e6-8000-82f100605a70 ...,snowplowanalytics.com,2021-12-02 08:09:00,snowplowanalytics.com,1.00
1372,07f53503-a5fa-485e-8b73-0d06f325dd34 ...,snowplowanalytics.com,2021-02-14 03:08:00,snowplowanalytics.com,1.00
3377,080453e8-f540-4221-8f70-124f264fd536 ...,snowplowanalytics.com,2021-12-02 10:46:00,snowplowanalytics.com,1.00
4854,0882d350-8bcf-4d5b-9da7-56bac6ef77b6 ...,snowplowanalytics.com,2021-02-13 14:27:00,snowplowanalytics.com,1.00
2639,090786d4-33e8-4068-83de-9e2db4e2947a ...,snowplowanalytics.com,2021-12-02 12:06:00,snowplowanalytics.com,1.00
1749,09a8f99c-35b5-4a41-9891-c3ad833f0364 ...,snowplowanalytics.com,2021-11-02 11:57:00,snowplowanalytics.com,1.00
712,0b601f39-2835-4c0c-a3c7-5139a32a1507 ...,snowplowanalytics.com,2021-11-02 09:05:00,snowplowanalytics.com,1.00
3194,0bf709f3-03c8-4721-87de-5fb89b632ca3 ...,snowplowanalytics.com,2021-12-02 16:00:00,snowplowanalytics.com,1.00
2213,0ccf115a-a3b8-4287-aac9-0cf9bc478fad ...,snowplowanalytics.com,2021-12-02 06:52:00,snowplowanalytics.com,1.00


In [246]:
# Create a table with all users going to with discourse.snowplowanalytics.com as second page_view
df_table2 = df_temp.loc[(df_temp['page_urlhost']=='discourse.snowplowanalytics.com')&(df_temp['rank']==2)]
# join both tables to find users starting on snowplowanalytics.com and then going to discourse.snowplowanalytics.com
df_merge = pd.merge(dfff1, dfff2, on=["session_id"])
# count distinct on session_id
numerator = len(df_merge)
len(df_merge)

20

In [247]:
percent_users_o2 = numerator/denominator
print(f'Distinct users starting on Snowplowanalytics: {denominator}')
print(f'Distinct users starting on Snowplowanalytics and continuing to Discourse: {numerator}')
print(f'Percentage of users is: {percent_users_o2:.2%}')

Distinct users starting on Snowplowanalytics: 2144
Distinct users starting on Snowplowanalytics and continuing to Discourse: 20
Percentage of users is: 0.93%


# Data analysis & modelling - Section 3

3. Imagine you intend to use this dataset for repeated analysis in the future, describe what transformations you would perform on the dataset to derive a set of views that contain analytics ready data. You do not need to write code defining your view(s).

   * I would include a <b>page_view_rank</b>. Simply ranking page_view in order of time appearance for each session on a given day. To calculate the Bounce Rate, I would simply need to count user with a max page_view_rank of 1.<br><br>
    
   * A page_view_rank can also be useful to recreate the full user journey by pivoting the ranks in columns containing the name of each page_view e.g. imagine a table with column= rank_1, value= snowplowanalytics.com. Essentially creating the customer journey on the website. It could help to follow patterns like: What % of users start their journey on page_urlhost ='snowplowanalytics.com' before navigating to page_urlhost='discourse.snowplowanalytics.com'. However, dvce_created_tstamp should include seconds to differentiate hits happening on same minute.<br><br>
    
   * I would also include a <b>session_duration</b> field. Essentially, tracking the duration of each active page view. It should be doable as a page_ping is sent at a regular cadence. The session_duration would make it much easier to compute the average time on a page.<br><br>
    
   * Similarly, I would also include a <b>scroll_depth</b> derived from the raw data.<br><br>
   
   * All the suggestions above aim to create metrics. They can be part of the raw data itself or can form a first aggregation table. A second aggregation layer could be to group metrics like session_duration or scroll_depth into buckets e.g. scroll_depth 25%, 50%, 75% and 100%.<br><br>
    
   * Part of this second aggregation layer, I would create a series of time entities derived from dvce_created_tstamp, such: Date (dd/mm/yyyy), Week, Month, Quarter, Year. It would be then much easier to aggregate any metric according to specific date dimensions and follow their evolution through time e.g. MoM, YoY.<br><br>